In [1]:
import csv

In [13]:
with open('MLTempDataset1.csv', 'r') as csv_file:
    csv_reader = csv.DictReader(csv_file)

    with open('time_series_temp.csv', 'w') as temp_series:
        fieldnames = ['Datetime', 'Hourly_Temp']
        
        csv_writer = csv.DictWriter(temp_series, fieldnames=fieldnames)
        csv_writer.writeheader()

        for line in csv_reader:
            del line['']
            csv_writer.writerow(line)
    
    